In [1]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [2]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))   # input(3,28,28) output(6,24,24)
        x = self.pool1(x)           # output(6,12,12)
        x = F.relu(self.conv2(x))   # output(16,8,8)
        x = self.pool2(x)           # output(16,4,4)
        # print(x.size())
        x = x.view(-1, 16 * 4 * 4)  # output(4*4*16)
        x = F.relu(self.fc1(x))     # output(120)
        x = F.relu(self.fc2(x))     # output(84)
        x = self.fc3(x)             # output(10)
        return x

In [3]:
# device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Lambda(lambda x: x.repeat(3,1,1)),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.MNIST(root='../data', train=True,
                                         download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=36,
                                           shuffle=False, num_workers=0)
val_set = torchvision.datasets.MNIST(root='../data', train=False,
                                       download=True, transform=transform)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=5000,
                                         shuffle=False, num_workers=0)
val_data_iter = iter(val_loader)
val_image, val_label = val_data_iter.next()
print(val_image.size())

net = LeNet()
net.to(device)
# 损失函数
loss_function = nn.CrossEntropyLoss()
# 定义优化器
optimizer = optim.Adam(net.parameters(), lr=0.001)

# 训练过程
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0 #累加损失
    for step, data in enumerate(train_loader, start=0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # print(inputs.size(), labels.size())
        # zero the parameter gradients
        optimizer.zero_grad() # 如果不清除历史梯度，就会对计算的历史梯度进行累加
        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = loss_function(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if step % 500 == 499:    # print every 500 mini-batches
            with torch.no_grad():
                outputs = net(val_image.to(device))  # [batch, 10]
                predict_y = torch.max(outputs, dim=1)[1]
                accuracy = (predict_y == val_label.to(device)).sum().item() / val_label.size(0)

                print('[%d, %5d] train_loss: %.3f  test_accuracy: %.3f' %
                      (epoch + 1, step + 1, running_loss / 100, accuracy))
                running_loss = 0.0

print('Finished Training')

save_path = '../model/Lenet.pth'
torch.save(net.state_dict(), save_path)

cuda:0
torch.Size([5000, 3, 28, 28])
[1,   500] train_loss: 2.108  test_accuracy: 0.949
[1,  1000] train_loss: 0.558  test_accuracy: 0.968
[1,  1500] train_loss: 0.519  test_accuracy: 0.977
[2,   500] train_loss: 0.371  test_accuracy: 0.973
[2,  1000] train_loss: 0.284  test_accuracy: 0.976
[2,  1500] train_loss: 0.324  test_accuracy: 0.980
[3,   500] train_loss: 0.251  test_accuracy: 0.978
[3,  1000] train_loss: 0.199  test_accuracy: 0.978
[3,  1500] train_loss: 0.265  test_accuracy: 0.981
[4,   500] train_loss: 0.182  test_accuracy: 0.978
[4,  1000] train_loss: 0.172  test_accuracy: 0.982
[4,  1500] train_loss: 0.208  test_accuracy: 0.982
[5,   500] train_loss: 0.148  test_accuracy: 0.980
[5,  1000] train_loss: 0.126  test_accuracy: 0.981
[5,  1500] train_loss: 0.175  test_accuracy: 0.983
[6,   500] train_loss: 0.112  test_accuracy: 0.981
[6,  1000] train_loss: 0.111  test_accuracy: 0.980
[6,  1500] train_loss: 0.144  test_accuracy: 0.981
[7,   500] train_loss: 0.112  test_accuracy: 

In [4]:
test_set = torchvision.datasets.MNIST(root='../data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10000,
                                         shuffle=False, num_workers=0)
test_data = next(iter(test_loader))

net = LeNet()
net.load_state_dict(torch.load('../model/lenet.pth'))
with torch.no_grad():
    x,y = test_data[0],test_data[1]
    output = net(x)
    pred = torch.max(output,1)[1]
    print(f'test acc:{sum(pred==y)/output.shape[0]}')

test acc:0.9890000224113464
